In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

Using gpu device 0: GeForce 710M (CNMeM is enabled with initial size: 85.0% of memory, cuDNN not available)


In [3]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

In [4]:
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

## Setup

In [5]:
#path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open("duda.txt").read().lower()
print('corpus length:', len(text))

corpus length: 131264


In [6]:
!tail {path} -n25

Nazwa 'tail' nie jest rozpoznawana jako polecenie wewn©trzne lub zewn©trzne,
program wykonywalny lub plik wsadowy.


In [8]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 70


In [9]:
chars.insert(0, "\0")

In [10]:
''.join(chars[1:-6])

'\n !"%()+,-.0123456789:?abcdefghijklmnopqrstuvwxyz\x84\x8c\x92\x94\x96\x9c\x9f\xa3\xad\xaf\xb3\xb9\xbf\xc1'

In [11]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [12]:
idx = [char_indices[c] for c in text]

In [13]:
idx[:10]

[42, 49, 24, 37, 38, 46, 37, 32, 2, 39]

In [14]:
''.join(indices_char[i] for i in idx[:70])

'szanowni pa\xf1stwo marsza\xb3kowie i marsza\xb3kowie seniorzy,\nszanowna pani p'

## Preprocess and create model

In [15]:
maxlen = 8
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen+1):
    sentences.append(idx[i: i + maxlen])
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

nb sequences: 131257


In [16]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [17]:
sentences.shape, next_chars.shape

((131255L, 8L), (131255L, 8L))

In [18]:
n_fac = 24

In [19]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])    

In [21]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam() )

## Train

In [22]:
def print_example():
    seed_string="szanowny"
    for i in range(500):
        x=np.array([char_indices[c] for c in seed_string[-8:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [51]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=32, nb_epoch=1, verbose = 2)

Epoch 1/1
2029s - loss: 1.6267


In [52]:
print_example()

szanowny panie marszalku i szanowni zebrkowych go�cie - �adowszego nadlatemu ona niepodleg�e podosta� partnerskich tego, co 15  pzycy szanszej na najlepsze w stanie za polska nie by�o ju� o�wodnia czasem polsce polski. ale te� tar� polskiego wyz�r, �e przed na tym, kt�rzy dla naszych przysz�o�ci. nowoczesnym rzeczypospolitej, zapalczywy na wszystkim si� rozw�juc�w na innego najwa�niejse projektywnego ? prosi�. kt�ra prze�odowania � i przyj��em dzia�alno�� na pa�stwo m�wi�, to jak �udziom wielkie poblicznego. bardzo serdecznie dzi�kuj�


In [20]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=32, nb_epoch=1, verbose =2)

Epoch 1/1
2019s - loss: 0.9727


In [23]:
print_example()

szanowny panie marszalku i szanowni zebra�y o tym, czy jakiekolwiek inicjatyw�, kt�rzy s� dotkni�ci chrzony polskich rodzin, cho�by fundamentalne znaczeniu � z ca�� pewno�ci� wspania�ym tego konkursu, wypitnie i to mocno podkre�la�� i nast�pcami �yczy� temu, �e ona zosta�y ustanowego by�a wtedy, �e jeszcze raz z panem prezydencki,
doskona�y przyznawana jest tu, de tak�e jeszcze raz dzi�kuj� za to, �e on jest najlepsze jest silne i sprawiedliwo�ci, zapewniam swoj� spo�ecze�stwo w czo��wce.
 
tak by�o w ci�gu ostatnich jak po to, �eby u


In [24]:
model.optimizer.lr=0.001

In [25]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1, verbose = 2)

Epoch 1/1
1624s - loss: 0.8839


In [26]:
print_example()

szanowny panie marszalku i szanowni zebranicznych.
 
po drugie, �eby by� wolny�.
 
to trzeba wsp�praca" si� rozwija� w jakim� sensie to one by�y pouczaj�ce i s�czeg�ln� do pierwszego pa�stwa. tak naprawd� w polsce. chcemy pami�ta�. mo�emy m�wi� �iawieli i kt�rego pocz�tku", �e ta polska r�wnie� z pewnego naszego kraju i odwagi i tej przyja�ni ? o jednym: musz� pa�stwa. tak jak po prostu j� w produkcja wstali� ca�y czas nas � cho�by w polsce jest wielkie wzmocnienie naszego kraju, a tak�e wiele sprawami nad zwyk�ym wolno�� i wydoby� t


In [27]:
model.optimizer.lr=0.0001

In [28]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1, verbose = 2)

Epoch 1/1
1658s - loss: 0.8479


In [29]:
print_example()

szanowny panie marszalku i szanowni zebralonej �wiatowej, bo tak jak m�wi�em wspierali�my jej suwerenno��. my�l� tutaj jest najlepiej dla nich nie wyda� najbli�si.
 
dlatego tak jak najbardziej prezydenta rzeczypospolitej bra� cz�onkostwo w polsce jest ogromnie to, co zawsze dla nich by�o odnowicielom sojuszu, buduj�c przede wszystkim bardzo cz�sto m�wimy ? kt�ry - najlepiej dla mnie wsp�lnie jest taka polska trybuna� w�gierskich miastach. to, prosz� pa�stwa, z naszego kraju jest tego, co nazywamy pewne obiektywne kryterii, kt�re ju� 


In [242]:
model.save_weights('data/char_rnn.h5')

In [33]:
model.optimizer.lr=0.00001

In [34]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1, verbose = 2)

Epoch 1/1
1656s - loss: 0.7968


In [35]:
print_example()

szanowny panie marszalku i szanowni zebralonalnych, si�ami w�adz regionalnych, sojuszniczych w tym konkursie, �eby by� w�a�nie z ozwarty na wnikomi, podobnym przez naszym wielkim zainteresowaniem izbysowych i wielkiego,
szanowni pa�stwo.
 
pa�stwa zak�ady poprzed tym, kt�rzy przyprowadzili�my do piel�gnowa�o si� do funkcjonuj�ca firm, fierzo tak�e jest tak,, �e mogli�my usi���, porozmawia� w walce o budowanie lepszej rzeczypospolitej szczyt nato w zakresie promowanie wielkie firmy b�d�ce efektem tego mojego wyst�pienia, dzi�kuj�.
szcz


In [36]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1, verbose = 2)

Epoch 1/1
2137s - loss: 0.7775


In [55]:
print_example()

szanowny panie marszalku i szanowni zebrani go�cie!
 
to dla mnie przede wszystkim tego, co zbudowa�o w ci�gu ? mo�liwo�ci �yciowe zosta�y wyr�wnane. pa�stwo pokoleniem jakby moich start-upowego, zdaj� sobie rad�, a kt�rej ludzko�� � mam nadziej� � b�dzie nale�a�a coraz wi�cej firm, sp�ek skarbu pa�stwa, dla mnie ogromnie pan b�g b�ogos�


In [37]:
model.save_weights('data/char_rnnduda1-2.h5')